# PREDOMINATELY TAKEN FROM https://github.com/SyneRBI/SIRF-Exercises/blob/master/notebooks/Synergistic/BrainWeb.ipynb

In [ ]:
!pip install brainweb
!pip install pyellipsoid

In [ ]:
import brainweb
import numpy as np
import matplotlib.pyplot as plt
url = r"http://brainweb.bic.mni.mcgill.ca/cgi/brainweb1?do_download_alias=subject04_crisp&format_value=raw_short&zip_value=gnuzip&download_for_real=%5BStart+download%21%5D"
fname = "subject_04.bin.gz"
files = brainweb.get_file(fname, url, "brainweb_3D/")
fname = "brainweb_3D/" + fname
data = brainweb.load_file(fname)
brainweb.seed(1337)
# You have to change the brainweb source to allow the outres of MR (1,1,1) mm
# see https://github.com/casperdcl/brainweb/issues/12 we chose to take ceil
vol = brainweb.get_mmr_fromfile(
    fname,
    petNoise=1, t1Noise=0.75, t2Noise=0.75,
    petSigma=1, t1Sigma=1, t2Sigma=1, outres = "MR")
vol_amyl = brainweb.get_mmr_fromfile(
    fname,
    petNoise=1, t1Noise=0.75, t2Noise=0.75,
    petSigma=1, t1Sigma=1, t2Sigma=1,
    PetClass=brainweb.Amyloid, outres = "MR")

FDG_arr  = vol['PET']
amyl_arr = vol_amyl['PET']
uMap_arr = vol['uMap']
T1_arr   = vol['T1']
T2_arr   = vol['T2']

for vol_path in ["brainweb_3D/subject_04.FDG.MR.npz","brainweb_3D/subject_04.Amyloid.MR.npz"]:
    vol = np.load(vol_path)
    vol_new = {}
    for vol_file in vol.files:
        vol_new[vol_file] = vol[vol_file]
    for vol_type in ["PET","uMap","T1","T2"]:
        vol_new[vol_type] = vol_new[vol_type][38:218, 231:-231, 231:-231]
    np.save(vol_path[:-3]+"ground_truth", vol_new)

def subplot_(idx,vol,title,clims=None,cmap="viridis"):
    plt.subplot(*idx)
    plt.imshow(vol,cmap=cmap)
    if not clims is None:
        plt.clim(clims)
    plt.colorbar()
    plt.title(title)
    plt.axis("off")

fdg = np.load("brainweb_3D/subject_04.FDG.MR.ground_truth.npy", allow_pickle=True).item()
amyl = np.load("brainweb_3D/subject_04.Amyloid.MR.ground_truth.npy", allow_pickle=True).item()
plt.figure();
slice_show = 100
subplot_([2,3,1],fdg["PET"][slice_show],'FDG',cmap="hot")
subplot_([2,3,2],amyl["PET"][slice_show],'Amyloid',cmap="hot")
subplot_([2,3,3],fdg["uMap"][slice_show],'uMap',cmap="bone")
subplot_([2,3,4],fdg["T1"][slice_show],'T1',cmap="Greys_r")
subplot_([2,3,5],fdg["T2"][slice_show],'T2',cmap="Greys_r")
print("The resolution is: ", vol["res"], "mm. The dimensions are: ", FDG_arr.shape, "voxels.")
print("Keys for the dicts: ", fdg.keys())

In [ ]:

import numpy as np
from skimage.filters import gaussian
import cupy as cp
import cupyx.scipy.ndimage as ndi
from pyellipsoid import drawing
import torch

for path in ["brainweb_3D/subject_04.FDG.MR.ground_truth.npy", "brainweb_3D/subject_04.Amyloid.MR.ground_truth.npy"]:
    dict_gt = np.load(path, allow_pickle=True).item()
    pet_vol = dict_gt["PET"]
    seg = cp.zeros_like(pet_vol)
    foreground, background = 1, 0
    seg[dict_gt["uMap"] <= 0] = background
    seg[dict_gt["uMap"] > 0] = foreground
    seg = ndi.binary_opening(seg, structure=cp.ones((3,3,3)))
    seg = ndi.binary_closing(seg, structure=cp.ones((20,20,20)), border_value=1)
    mask = seg.copy()
    for _ in range(15):
        mask = ndi.binary_erosion(mask, border_value=1)
    mask_np = mask.get()

    tumours_dict = {}
    # Define an image shape, axis order is: Z, Y, X
    image_shape = mask_np.shape

    # Define an image shape, axis order is: Z, Y, X
    image_shape = mask_np.shape
    # Define an ellipsoid, axis order is: X, Y, Z
    ell_center = [150,182,110]
    ell_radii = (8, 19, 9)
    ell_angles = (1231, 12312, 23432) # Order of rotations is X, Y, Z
    # Draw a 3D binary image containing the ellipsoid
    tumours_dict["tumour_mask_0"] = drawing.make_ellipsoid_image(image_shape, ell_center, ell_radii, ell_angles)

    # Define an ellipsoid, axis order is: X, Y, Z
    ell_center = [94,62,95]
    ell_radii = (15, 5, 10)
    ell_angles = (213,40,60) # Order of rotations is X, Y, Z
    # Draw a 3D binary image containing the ellipsoid
    tumours_dict["tumour_mask_1"] = drawing.make_ellipsoid_image(image_shape, ell_center, ell_radii, ell_angles)

    # Define an image shape, axis order is: Z, Y, X
    image_shape = mask_np.shape
    # Define an ellipsoid, axis order is: X, Y, Z
    ell_center = [102,162,25]
    ell_radii = (12, 14, 8)
    ell_angles = (0, 0, 0) # Order of rotations is X, Y, Z
    # Draw a 3D binary image containing the ellipsoid
    tumours_dict["tumour_mask_2"] = drawing.make_ellipsoid_image(image_shape, ell_center, ell_radii, ell_angles)

    # Define an image shape, axis order is: Z, Y, X
    image_shape = mask_np.shape
    # Define an ellipsoid, axis order is: X, Y, Z
    ell_center = [93,122,62]
    ell_radii = (16, 8, 12)
    ell_angles = (123,21,53) # Order of rotations is X, Y, Z
    # Draw a 3D binary image containing the ellipsoid=
    tumours_dict["tumour_mask_3"] = drawing.make_ellipsoid_image(image_shape, ell_center, ell_radii, ell_angles)

    # Define an image shape, axis order is: Z, Y, X
    image_shape = mask_np.shape
    # Define an ellipsoid, axis order is: X, Y, Z
    ell_center = [142,112,142]
    ell_radii = (9, 10, 6)
    ell_angles = (1231, 12312, 23432) # Order of rotations is X, Y, Z
    # Draw a 3D binary image containing the ellipsoid
    tumours_dict["tumour_mask_4"] = drawing.make_ellipsoid_image(image_shape, ell_center, ell_radii, ell_angles)


    intensities = [0.78,.76,1.06,.89,.92]
    tumours = np.zeros_like(pet_vol)
    backgrounds = np.zeros_like(pet_vol)
    for i in range(5):
        tumour_mask = tumours_dict["tumour_mask_"+str(i)].astype(float)
        outer = cp.array(tumour_mask)
        for _ in range(15):
            outer = ndi.binary_dilation(outer,border_value=0)
        inner = cp.array(tumour_mask)
        for _ in range(8):
            inner = ndi.binary_dilation(inner, border_value=0)
        background_mask = mask_np*(outer.get().astype(float)-inner.get().astype(float))
        tumours_dict["background_mask_"+str(i)] = background_mask
        tumours += gaussian(tumour_mask, sigma=1)*intensities[i]*pet_vol.max()
        backgrounds += background_mask

    pet_tumours = np.max((pet_vol, tumours), axis=0)

    vol = np.load(path, allow_pickle=True).item()
    vol["tumours_dict"] = tumours_dict
    vol["PET_tumours"] = pet_tumours
    np.save(path[:-3]+"tumours", vol)

    """ for i in range(len(pet_vol)):#unique_axial_coords:
        plt.figure(figsize=(15,5));
        slice_show = i
        subplot_([1,3,1], pet_tumours[slice_show], 'FDG', cmap = "hot")
        subplot_([1,3,2], mask_np[slice_show], 'mask', cmap="hot")
        subplot_([1,3,3], tumours[slice_show]/tumours.max() + backgrounds[slice_show], 'tumour+background', cmap="hot")
        plt.show() """
    print("The resolution is: ", vol["res"], "mm. The dimensions are: ", pet_tumours.shape, "voxels.")


In [ ]:
from sirf.Utilities import examples_data_path
import sirf.STIR as pet
pet.set_verbosity(0)
pet.AcquisitionData.set_storage_scheme("memory")
pet.MessageRedirector(info=None, warn=None, errr=None)
import numpy as np

for path in ["brainweb_3D/subject_04.FDG.MR.ground_truth.tumours.npy", "brainweb_3D/subject_04.Amyloid.MR.ground_truth.tumours.npy"]:
    gt_tumour_dict = np.load(path, allow_pickle=True).item()

    mMR_template_sino = examples_data_path('PET') + "/mMR/mMR_template_span11.hs"
    mMR_template = pet.AcquisitionData(mMR_template_sino)
    image = mMR_template.create_uniform_image(1.0)
    image_hr = image.zoom_image(
                    zooms=(2.03125/1, 2.08626/1, 2.08626/1),
                    offsets_in_mm=(0., 0., 0.),
                    size=(180, 256, 256))
    image_hr = image_hr.move_to_scanner_centre(mMR_template)
    image_hr.fill(gt_tumour_dict["PET_tumours"])
    image_lr = image_hr.zoom_image(
                zooms=(1/2, 1/2, 1/2),
                offsets_in_mm=(0., 0., 0.),
                size=(90,128,128))
    image_lr = image_lr.move_to_scanner_centre(mMR_template)

    print("Image original shape: ", image.shape, ". Voxel dimensions (mm): ", image.voxel_sizes())
    print("Image high resolution shape: ", image_hr.shape, ". Voxel dimensions (mm): ", image_hr.voxel_sizes())
    print("Image low resolution shape: ", image_lr.shape, ". Voxel dimensions (mm): ", image_lr.voxel_sizes())

    # HIGH RESOLUTION
    am = pet.AcquisitionModelUsingParallelproj()
    # Set up sensitivity due to attenuation
    uMap = image_hr.clone().fill(gt_tumour_dict["uMap"])
    asm_attn = pet.AcquisitionSensitivityModel(uMap, am)
    asm_attn.set_up(mMR_template)
    bin_eff_hr = mMR_template.get_uniform_copy(1.)
    print('applying attenuation (please wait, may take a while)...')
    asm_attn.unnormalise(bin_eff_hr)
    asm = pet.AcquisitionSensitivityModel(bin_eff_hr)
    am.set_acquisition_sensitivity(asm)
    am.set_up(mMR_template, image_hr)
    clean_measurements_hr = am.forward(image_hr)

    
    # HIGH RESOLUTION
    am = pet.AcquisitionModelUsingParallelproj()
    # Set up sensitivity due to attenuation
    uMap = image_hr.clone().fill(gt_tumour_dict["uMap"])
    asm_attn = pet.AcquisitionSensitivityModel(uMap, am)
    asm_attn.set_up(mMR_template)
    bin_eff_hr = mMR_template.get_uniform_copy(1.)
    print('applying attenuation (please wait, may take a while)...')
    asm_attn.unnormalise(bin_eff_hr)
    asm = pet.AcquisitionSensitivityModel(bin_eff_hr)
    am.set_acquisition_sensitivity(asm)
    am.set_up(mMR_template, image_lr)
    clean_measurements_lr = am.forward(image_lr)

    print("Rescaling for change image size with gt efficiencies: ", clean_measurements_lr.sum()/clean_measurements_hr.sum())

    name = "brainweb_3D/"
    if "Amyloid" in path:
        name += "Amyloid_"
    else:
        name += "FDG_"

    image_hr.write(name+"PET_hr")
    clean_measurements_hr.write(name+"clean_measurements_hr")
    bin_eff_hr.write(name+"bin_eff_hr")

    image_lr.write(name+"PET_lr")


In [ ]:
import matplotlib.pyplot as plt
import torch
pet_old = torch.load("path_to/src/brainweb_2d/clean/clean_test.pt")["reference"].detach().cpu().numpy()
image_fdg = pet.ImageData("path_to/src/sirf/brainweb_3D/FDG_PET_lr.hv")
image_amyl = pet.ImageData("path_to/src/sirf/brainweb_3D/Amyloid_PET_lr.hv")
fig, ax = plt.subplots(1,3,figsize=(15,5))
slice_show = 30
# array manipulation .swapaxes(0,1)[::-1,::-1]
fig.colorbar(ax[0].imshow(pet_old[slice_show,0,...].swapaxes(0,1)[::-1,::-1]))
ax[0].set_title("Original")
fig.colorbar(ax[1].imshow(image_fdg.as_array()[slice_show,...]))
ax[1].set_title("FDG")
fig.colorbar(ax[2].imshow(image_amyl.as_array()[slice_show,...]))
ax[2].set_title("Amyloid")
pet_slice = pet_old[slice_show,0,...]
plt.show()
print(np.sum(pet_slice.swapaxes(0,1)[::-1,::-1].swapaxes(0,1)[::-1,::-1]-pet_slice)) 

# Validate that the old processed data is similar orientation/space as new data
import torch
pet_old = torch.load("path_to/src/brainweb_2d/clean/clean_test.pt")["reference"].detach().cpu().numpy()
image_fdg = pet.ImageData("path_to/src/sirf/brainweb_3D/FDG_PET_hr.hv")
image_amyl = pet.ImageData("path_to/src/sirf/brainweb_3D/Amyloid_PET_hr.hv")
fig, ax = plt.subplots(1,3,figsize=(15,5))
# array manipulation .swapaxes(0,1)[::-1,::-1]
fig.colorbar(ax[0].imshow(pet_old[slice_show,0,...].swapaxes(0,1)[::-1,::-1]))
ax[0].set_title("Original")
fig.colorbar(ax[1].imshow(image_fdg.as_array()[2*slice_show,...]))
ax[1].set_title("FDG")
fig.colorbar(ax[2].imshow(image_amyl.as_array()[2*slice_show,...]))
ax[2].set_title("Amyloid")
pet_slice = pet_old[slice_show,0,...]
plt.show()
print(np.sum(pet_slice.swapaxes(0,1)[::-1,::-1].swapaxes(0,1)[::-1,::-1]-pet_slice))


In [ ]:
for path in ["brainweb_3D/subject_04.FDG.MR.ground_truth.tumours.npy", "brainweb_3D/subject_04.Amyloid.MR.ground_truth.tumours.npy"]:
    gt_tumour_dict = np.load(path, allow_pickle=True).item()
    tumour_dict = gt_tumour_dict["tumours_dict"]
    name = "brainweb_3D/"
    if "Amyloid" in path:
        name += "Amyloid_"
    else:
        name += "FDG_"
    for i in range(5):
        for masks in ["tumour_mask", "background_mask"]:
            mask = tumour_dict[masks+"_"+str(i)]
            mMR_template_sino = examples_data_path('PET') + "/mMR/mMR_template_span11.hs"
            mMR_template = pet.AcquisitionData(mMR_template_sino)
            image = mMR_template.create_uniform_image(1.0)
            image_hr = image.zoom_image(
                            zooms=(2.03125/1, 2.08626/1, 2.08626/1),
                            offsets_in_mm=(0., 0., 0.),
                            size=(180, 256, 256))
            image_hr = image_hr.move_to_scanner_centre(mMR_template)
            image_hr.fill(mask)
            image_lr = image_hr.zoom_image(
                        zooms=(1/2, 1/2, 1/2),
                        offsets_in_mm=(0., 0., 0.),
                        size=(90,128,128))
            image_lr = image_lr.move_to_scanner_centre(mMR_template)
            image_lr.write(name+masks+"_"+str(i))

In [ ]:
import sirf.STIR as pet
import torch
tumours = []
backgrounds = []
for i in range(5):
    tumour = pet.ImageData(("brainweb_3D/Amyloid_tumour_mask_"+str(i))+".hv").as_array()
    background = pet.ImageData(("brainweb_3D/Amyloid_background_mask_"+str(i))+".hv").as_array()
    tumours.append(torch.from_numpy(tumour))
    backgrounds.append(torch.from_numpy(background))
tumours = torch.stack(tumours)
backgrounds = torch.stack(backgrounds)
torch.save(tumours, "brainweb_3D/tumours.pt")
torch.save(backgrounds, "brainweb_3D/backgrounds.pt")

In [ ]:
# Uncomment to generate noisy data
import sirf.STIR as pet
pet.set_verbosity(0)
from sirf.Utilities import examples_data_path
pet.AcquisitionData.set_storage_scheme("memory")
pet.MessageRedirector(info=None, warn=None, errr=None)
import os
os.makedirs("brainweb_3D/noisy", exist_ok=True)
import numpy as np
num_realisations = 5
np.random.seed(42)
for tracer in ["Amyloid", "FDG"]:
    for num_counts in [4e7]:
        for realisation in range(num_realisations):
            name = "brainweb_3D/noisy/" + tracer + "_noisy_measurements_" + "{:.0e}".format(num_counts) + "_" + str(realisation)
            clean_measurements = pet.AcquisitionData("brainweb_3D/" + tracer + "_clean_measurements_hr.hs")
            clean_measurements_arr = np.abs(clean_measurements.as_array())
            print("Noise rescaling factor: ", num_counts/clean_measurements_arr.sum())
            clean_measurements_arr = clean_measurements_arr/clean_measurements_arr.sum()*num_counts
            noisy_measurements_arr = np.random.poisson(clean_measurements_arr).astype('float32')
            noisy_measurements = clean_measurements.get_uniform_copy()
            noisy_measurements.fill(noisy_measurements_arr)
            noisy_measurements.write(name)


In [ ]:
# Get 1 epoch OSEM images for noises and realisations
import sirf.STIR as pet
import numpy as np
pet.set_verbosity(0)
from sirf.Utilities import examples_data_path
pet.AcquisitionData.set_storage_scheme("memory")
pet.MessageRedirector(info=None, warn=None, errr=None)
num_realisations = 5
num_subsets = 28
num_epochs = 1
for tracer in ["Amyloid", "FDG"]:
    image = pet.ImageData(f"brainweb_3D/{tracer}_PET_lr.hv")
    bin_eff = pet.AcquisitionData(f"brainweb_3D/{tracer}_bin_eff_hr.hs")
    data_template = bin_eff.get_uniform_copy(1.)
    image_template = image.get_uniform_copy(1.)
    am = pet.AcquisitionModelUsingParallelproj()
    asm = pet.AcquisitionSensitivityModel(bin_eff)
    am.set_acquisition_sensitivity(asm)
    am.set_up(data_template, image.get_uniform_copy(1))
    sens_image = am.backward(data_template)
    sens_image.write(f"brainweb_3D/{tracer}_sensitivity_image.hv")
    sens_image_np = sens_image.as_array()
    views = data_template.shape[2]
    acquisition_models_sirf = []
    for i in range(num_subsets):
        subset_idxs = list(range(views))[i:][::num_subsets]
        bin_eff_subset = bin_eff.get_subset(subset_idxs)
        data_template_subset = bin_eff_subset.get_uniform_copy(1.)
        # SET UP THE ACQUISITION MODEL
        acquisition_model = pet.AcquisitionModelUsingParallelproj()
        sensitivity_factors = pet.AcquisitionSensitivityModel(bin_eff_subset)
        acquisition_model.set_acquisition_sensitivity(sensitivity_factors)
        acquisition_model.set_up(data_template_subset, image)
        acquisition_models_sirf.append(acquisition_model)
        print("Acquisition model", i+1, "of", num_subsets, "set up")
    for num_counts in [4e7]:
        for realisation in range(num_realisations):
            name = "brainweb_3D/noisy/" + tracer + "_noisy_measurements_" + "{:.0e}".format(num_counts) + "_" + str(realisation)
            noisy_measurements = pet.AcquisitionData(name + ".hs")
            objectives_sirf = []
            for i in range(num_subsets):
                subset_idxs = list(range(views))[i:][::num_subsets]
                objective_sirf = pet.make_Poisson_loglikelihood(noisy_measurements.get_subset(subset_idxs), acq_model = acquisition_models_sirf[i])
                objective_sirf.set_up(image)
                objectives_sirf.append(objective_sirf)
                print("Objective function", i+1, "of", num_subsets, "set up")
            x = image.get_uniform_copy(1)
            views = noisy_measurements.shape[2]
            for j in range(num_subsets):
                bp_diff = objectives_sirf[j].get_gradient(x)
                x = x + num_subsets * ((x+1e-6)/sens_image)*bp_diff
                x_np = x.as_array()
                x_np[sens_image_np==0] = 0
                x_np[x_np<0] = 0
                x.fill(x_np)
            print("OSEM ", tracer, num_counts, realisation, " done")
            x.write("brainweb_3D/noisy/" + tracer + "_osem_" + "{:.0e}".format(num_counts) + "_" + str(realisation))

In [ ]:
import sirf.STIR as pet
pet.set_verbosity(0)
from sirf.Utilities import examples_data_path
pet.AcquisitionData.set_storage_scheme("memory")
import numpy as np
num_subsets = 28
num_epochs = 10
noise = "4e+07"
tracer = "Amyloid"
image = pet.ImageData(f"brainweb_3D/{tracer}_PET_lr.hv")
measurements = pet.AcquisitionData(f"brainweb_3D/noisy/{tracer}_noisy_measurements_{noise}_0.hs")
bin_eff = pet.AcquisitionData(f"brainweb_3D/{tracer}_bin_eff_hr.hs")

am = pet.AcquisitionModelUsingParallelproj()
asm = pet.AcquisitionSensitivityModel(bin_eff)
am.set_acquisition_sensitivity(asm)
am.set_up(measurements, image.get_uniform_copy(1))
sens_image = am.backward(measurements.get_uniform_copy(1.))
views = measurements.shape[2]
noisy_measurements_subsets_sirf = []
acquisition_models_sirf = []
objectives_sirf = []
for i in range(num_subsets):
    subset_idxs = list(range(views))[i:][::num_subsets]
    # Get subset of data
    noisy_measurements_sirf = measurements.get_subset(subset_idxs)
    bin_eff_subset = bin_eff.get_subset(subset_idxs)

    # List noisy subsets
    noisy_measurements_subsets_sirf.append(noisy_measurements_sirf)

    # SET UP THE ACQUISITION MODEL
    sensitivity_factors = pet.AcquisitionSensitivityModel(bin_eff_subset)
    acquisition_model = pet.AcquisitionModelUsingParallelproj()
    acquisition_model.set_acquisition_sensitivity(sensitivity_factors)
    acquisition_model.set_up(noisy_measurements_subsets_sirf[i], image)
    acquisition_models_sirf.append(acquisition_model)
    objective_sirf = pet.make_Poisson_loglikelihood(noisy_measurements_subsets_sirf[i], acq_model = acquisition_models_sirf[i])
    objective_sirf.set_up(image)
    objectives_sirf.append(objective_sirf)
    print("Acquisition model", i+1, "of", num_subsets, "set up")


In [ ]:
x = image.get_uniform_copy(1)
# INITIALISE THE RECONSTRUCTION OBJECT
import matplotlib.pyplot as plt
sens_image_np = sens_image.as_array()
for i in range(10):
    print(f"Epoch {i+1}")
    for j in range(num_subsets):
        bp_diff = objectives_sirf[j].get_gradient(x)
        x = x + num_subsets * ((x+1e-6)/sens_image)*bp_diff
        x_np = x.as_array()
        x_np[sens_image_np==0] = 0
        x_np[x_np<0] = 0
        x.fill(x_np)
    plt.imshow(x.as_array()[30])
    plt.colorbar()
    plt.show()